# Research Extension

We observe from our results of generating decision trees with the ID3 algorithm on the spam, volcanoes, and voting datasets that the accuracy of the decision tree on our test data does not increase at higher depths.

We hypothesize that this is due to the fact that ID3 is a greedy algorithm that picks split attributes with the highest information gain at each partition, and therefore generates trees with higher information gain for shorter trees and near the root, but does not necessarily generate an accurate discriminator of the dataset at higher depth values.

In order to improve the accuracy of the ID3 algorithm on decision trees of higher depths, we employed the following strategies:

1. Weight towards split attributes that generate partitions of equal size
2. Stochastic Split - implement an element of randomness 



# 1. Weight towards split attributes that generate partitions of equal size

We hypothesize that the accuracy ID3 decision trees do not improve at higher depths is due to greedy selection of high information gain attributes at the root. This is observed by the first split feature of `os` on the spam dataset and `image_id` on the volcanoes dataset, which generate partitions which high information gain, but are not sensible decision boundaries for the target concept.

While split attributes with higher information gain are desirable in order to generate partitions with low class label entropy or pure nodes, ID3's greedy selection of the attribute with highest information gain creates a bias towards attributes with multiple values and high entropy. This is particularly observed by the selection of `image_id` on the volcanoes dataset.

This behavior can be compensated for by using gain ratio instead of information gain as the criterion for selecting split attributes.
$$ GainRatio(X) = \frac{InfoGain(X)}{Entropy(X)} $$


### Results of Using GainRatio Split Criteria

Below are our results of using gain ratio as our split criteria on the `volcanoes` dataset

- Depth 2
```
Accuracy:0.67
Size:  5
Maximum Depth:  2
First Feature:  chip_ 7_ 5
```

- Depth 4
```
Accuracy:0.72
Size:  274
Maximum Depth:  4
First Feature:  chip_ 7_ 5
```

- Depth 8
```
Accuracy:0.72
Size:  626
Maximum Depth:  8
First Feature:  chip_ 7_ 5
```

- Depth 16
```
Accuracy:0.72
Size:  634
Maximum Depth:  9
First Feature:  chip_ 7_ 5
```

- Depth 32
```
Accuracy:0.72
Size:  634
Maximum Depth:  9
First Feature:  chip_ 7_ 5
```

Although we can not observe an increase in the accuracy of our model at higher depths, we do observe that the our model selects a more sensible first split attribute `chip_ 7_ 5`, which corresponds to the pixel value of the volacano image.

We also observe that decision boundaries on image data such as the volcanoes dataset depends highly on the relationship between attributes, which decision trees can not express effectively.

Additionally, we observe that even at higher depth limits of 16 and 32, our algorithm generates decision trees of depth 9. This indicates that every leaf node of the decision tree is a pure node, and inaccuracies in predictions are due to overfitting, so pruning strategies could possibly yield higher accuracy on the dataset.


Below are our results of using gain ratio as our split criteria on the `spam` dataset

- Depth 2
```
Accuracy:0.65
Size:  5
Maximum Depth:  2
First Feature:  geoDistance
```

- Depth 4
```
Accuracy:0.74
Size:  49
Maximum Depth:  4
First Feature:  geoDistance
```

- Depth 8
```
Accuracy:0.76
Size:  698
Maximum Depth:  8
First Feature:  geoDistance
```

- Depth 16
```
Accuracy:0.82
Size:  8409
Maximum Depth:  16
First Feature:  3whs
```

- Depth 32
```
Accuracy:0.81
Size:  24516
Maximum Depth:  32
First Feature:  3whs
```

Like the `volcanoes` dataset, we do not observe an increase in the accuracy of our model using gain ratio in the `spam` dataset. However, we do observe that selected split features of `geoDistance` (Geographical distance between sender and ICSI) and `3whs` (Time between the arrival of the SYN from the remote host and arrival of ACK of the SYN/ACK sent by the local host.), are reasonably more sensible indicators of spam emails than `os` (OS of the remote host) selected by informatiaon gain.

# 1. Stochastic Split - Implementing an element of randomness to the split selection
While clearly doing a random selection all of the time would not lead to an accurate learning algorithm, and would lead to an accuracy of only about fifty percent, we believed that by having the option of choosing a split at random, it would avoid some of the biases of the algorithm by giving it different partitions to look at than it would have otherwise chosen.

For the purposes of this assignment we set the random rate at 30%, while this is particularly high we wanted to make sure that we would see the random split occur even on depths of five or similar depths that we were testing on. In a non-test setting a lower randomization rate would be preferred. Each time the algorithm goes to make a test is run and if the number is below thirty, it chooses the split criteria randomly, and if it is above thirty it follows the expected choice set up. The split itself was not the end of the method we implemented however.

Additionally, we implemented a method that would compare the forest of trees with these occasionally randomly generated partitions. Since the randomness can decrease accuracy if it occurs in the wrong place, we do not just let the decision tree with stochastic split train once. The algorithm runs five times and will print the tree with the highest accuracy metrics, training several times. This helps insure that the random split occurs at situations that help the processing of the tree, or in some cases where the data is optimized for an id3 tree already, may end up choosing one where a random split did not occur at all, but the test of the random split allows us to ensure that is the most accurate way to run it.

Here is what we found

2. An Analysis of Our Findings:
Ultimately we found that this method was not an effective way of solving the bias towards multi-value attributes. While the randomization would mitigate the bias when
it was put into effect, the counter effects of choosing randomly meant that it was not overall more accurate. Testing each class on a depth of four,

We found that the accuracies using the random method w/ depth four were as follows:
For volcanoes the best tree chose a different first test, and was slightly higher in accuracy over the one without randomization. It chose the first test chip_15_ 4 and ended with an accuracy of
.74, higher than the other accuracy of .73 without the randomization.

The best accuracy we found with this method for spam was 0.73 with a first feature of senderhour, which is worse than the accuracy of .74 without the randomization

Voting accuracy could also not be improved by using tests, but voting accuracy was already at 1 when the depth of the tree was four. Since voting only had three possible values for each feature, it was not as affected by this bias and so we did not expect voting to be heavily influenced by our changes

Based on these results, a random split method is not an effective way to reduce the multi-value bias. We realized an additional part of our accuracy method may fall in the pruning step. We found that even for tests with increased depth, overfitting happened very quickly.1
A random split was not going to be effective in reducing that problem, but that was not a problem we realized until after the testing occurred.
An additional method we would consider implementing in the future would be a random split that instead of choosing completely at random. While this method seemed like it could improve the multi-value attribute bias, it turned out to not be as effective in implementation as we hoped.